In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import os
import numpy as np
import math
import datetime
import json

In [2]:
path = '../taxi data/taxi_zones_with_latlong.csv'
taxiZone = pd.read_csv(path)

taxiZone = taxiZone.drop(columns=['OBJECTID','Shape_Leng', 'Shape_Area'])
taxiZone.head()

,zone,LocationID,borough,lat,long
0,Newark Airport,1,EWR,-74.176786,40.689516
1,Jamaica Bay,2,Queens,-73.826126,40.625724
2,Allerton/Pelham Gardens,3,Bronx,-73.849479,40.865888
3,Alphabet City,4,Manhattan,-73.977023,40.724152
4,Arden Heights,5,Staten Island,-74.189930,40.550340


In [3]:
path = '../taxi data/all data/october.csv'
october = pd.read_csv(path)

october["pickup"] = pd.to_datetime(october['pickup'])
october['pickup'] = october['pickup'].dt.date

october["dropoff"] = pd.to_datetime(october['dropoff'])
october['dropoff'] = october['dropoff'].dt.date


path = '../taxi data/all data/november.csv'
november = pd.read_csv(path)
november["pickup"] = pd.to_datetime(november['pickup'])
november['pickup'] = november['pickup'].dt.date

november["dropoff"] = pd.to_datetime(november['dropoff'])
november['dropoff'] = november['dropoff'].dt.date


path = '../taxi data/all data/december.csv'
december = pd.read_csv(path)
december["pickup"] = pd.to_datetime(december['pickup'])
december['pickup'] = december['pickup'].dt.date

december["dropoff"] = pd.to_datetime(december['dropoff'])
december['dropoff'] = december['dropoff'].dt.date

In [4]:
october.head()

,pickup,dropoff,pickupLocID,dropoffLocID
0,2018-10-01,2018-10-01,255.0,97.0
1,2018-10-01,2018-10-01,97.0,49.0
2,2018-10-01,2018-10-01,25.0,181.0
3,2018-10-01,2018-10-01,25.0,40.0
4,2018-10-01,2018-10-01,25.0,257.0


In [5]:
# exporting the whole month of october data to a json file
def getWholeData(allData, month, days):
    
    loc = taxiZone['LocationID'].to_list()
    zo = taxiZone['zone'].to_list()
    d = {'zone': zo, 'location': loc}
    wholeData = pd.DataFrame(data=d)

#     wholeData.head()
    for i in range(1, days + 1):
        date = datetime.date(2018, month, i)
    # #     print(date)
        dropdf = allData[(allData.dropoff == date)]
        dropdf = dropdf.rename(columns={'dropoff': 'date',
                                                       'dropoffLocID': 'LocationID'})
        dropdf = dropdf.merge(taxiZone, how='inner', on='LocationID')

        df = dropdf['zone'].value_counts().rename_axis('zone').reset_index(name='dropoffCount')

        df = df.merge(taxiZone,on='zone', how='inner')

        temp = []
        for index, row in wholeData.iterrows():
            locID = row["location"]

            if locID in df['LocationID'].values:
                val = df.loc[df['LocationID'] == locID, 'dropoffCount'].to_list()
    #             print(df.loc[df['LocationID'] == locID, 'dropoffCount'].to_list())
                temp.append(val[0])
            else:
                temp.append(0)

        wholeData[str(date)] = temp
        
    return wholeData
    
# wholeData.head()

octWhole = getWholeData(october,10, 31)
novWhole = getWholeData(november,11, 30)
decWhole = getWholeData(december,12, 31)

In [6]:
octWhole.head()

,zone,location,2018-10-01,2018-10-02,2018-10-03,2018-10-04,2018-10-05,2018-10-06,2018-10-07,2018-10-08,...,2018-10-22,2018-10-23,2018-10-24,2018-10-25,2018-10-26,2018-10-27,2018-10-28,2018-10-29,2018-10-30,2018-10-31
0,Newark Airport,1,4485,3778,4450,5780,5957,3958,5196,5055,...,4652,4089,4396,4918,5027,3100,5805,4976,4071,3817
1,Jamaica Bay,2,2,3,1,1,0,5,2,0,...,0,0,1,0,1,3,0,1,2,1
2,Allerton/Pelham Gardens,3,950,967,1001,1017,1092,1117,1027,848,...,953,935,998,1032,1168,1210,1055,941,928,1083
3,Alphabet City,4,2635,3101,3122,3400,3875,4932,4123,2774,...,2899,2968,3206,3565,4432,6675,4932,2840,3064,3371
4,Arden Heights,5,119,116,114,140,143,157,141,98,...,110,124,126,132,154,171,134,114,105,146


In [22]:

octWhole.to_csv('taxi data/all data/octoberWhole.csv', index=False)
novWhole.to_csv('taxi data/all data/novemberWhole.csv', index=False)
decWhole.to_csv('taxi data/all data/decemberWhole.csv', index=False)

In [8]:
def extractWhole(month, r, wholeData):
    
    resultDict = []
    columnList = wholeData.columns.to_list()
    # print(columnList)
    for index, row in wholeData.iterrows():

    #         resultDict[row['location']] = {}
        temp = {}
        temp["zone"] = row['zone']
        temp['location'] = row['location']
        temp['values'] = []

        for i in range(2,r):
            temp['values'].append({'date' : i-1, 'value': row[i]})

    #     print(temp)


        resultDict.append(temp)

    # print(resultDict)

    with open('output/{}/whole.json'.format(month), 'w') as fp:
        json.dump(resultDict, fp)
        
extractWhole('october', 33, octWhole)
extractWhole('november', 32, novWhole)
extractWhole('december', 33, decWhole)
        

In [13]:
# exporting the whole month of october data to json files
for i in range(1,32):
    date = datetime.date(2018, 12, i)
    print(date)
    dropdf = december[(december.dropoff == date)]
    dropdf = dropdf.rename(columns={'dropoff': 'date',
                                                   'dropoffLocID': 'LocationID'})
    dropdf = dropdf.merge(taxiZone, how='inner', on='LocationID')
    
    df = dropdf['zone'].value_counts().rename_axis('zone').reset_index(name='dropoffCount')

    df = df.merge(taxiZone,on='zone', how='inner')
    
    print(df.dropoffCount.max(), df.dropoffCount.min())
    
#     print(df)

    resultDict = {}
    for index, row in df.iterrows():
        resultDict[row['LocationID']] = row.to_dict()

    with open('output/december/{}.json'.format(date), 'w') as fp:
        json.dump(resultDict, fp)

2018-12-01
27235 1
2018-12-02
20675 3
2018-12-03
22644 2
2018-12-04
24164 1
2018-12-05
23646 5
2018-12-06
25258 2
2018-12-07
24856 1
2018-12-08
31170 2
2018-12-09
19610 4
2018-12-10
23711 1
2018-12-11
24011 2
2018-12-12
24735 3
2018-12-13
24695 3
2018-12-14
23321 2
2018-12-15
26719 2
2018-12-16
24045 2
2018-12-17
22427 1
2018-12-18
24074 1
2018-12-19
23577 1
2018-12-20
23296 1
2018-12-21
23441 3
2018-12-22
21264 1
2018-12-23
19808 19
2018-12-24
15805 1
2018-12-25
11523 1
2018-12-26
16122 1
2018-12-27
17359 1
2018-12-28
20575 1
2018-12-29
16176 1
2018-12-30
15186 2
2018-12-31
18187 1


In [10]:
# exporting max dropoff for each month
def exportMaxDrop(month, day, allData, name):
    resultDict = {}
    for i in range(1,day):
        date = datetime.date(2018, month, i)
    #     print(date)
        dropdf = allData[(allData.dropoff == date)]
        dropdf = dropdf.rename(columns={'dropoff': 'date',
                                                       'dropoffLocID': 'LocationID'})
        dropdf = dropdf.merge(taxiZone, how='inner', on='LocationID')

        df = dropdf['zone'].value_counts().rename_axis('zone').reset_index(name='dropoffCount')

        df = df.merge(taxiZone,on='zone', how='inner')

        resultDict["{}".format(date)] = df.dropoffCount.max()

        with open('output/{}/domain.json'.format(name), 'w') as fp:
            json.dump(resultDict, fp)
            
exportMaxDrop(10, 32, october, "october")
exportMaxDrop(11, 31, november, "november")
exportMaxDrop(12, 32, december, "december")

In [ ]:
diwali = datetime.date(2018, 10, 13)


diwaliDropoff = allData[(allData.dropoff == diwali)]
diwaliDropoff = diwaliDropoff.rename(columns={'dropoff': 'date',
                                                   'dropoffLocID': 'LocationID'})
print(diwaliDropoff.shape)
diwaliDropoff.tail()

In [ ]:
diwaliDropoff = diwaliDropoff.merge(taxiZone, how='inner', on='LocationID')
diwaliDropoff.head()

In [ ]:
diwaliDropoff['borough'].value_counts()

In [ ]:
diwaliDropoff['zone'].value_counts()

In [ ]:
df = diwaliDropoff['zone'].value_counts().rename_axis('zone').reset_index(name='dropoffCount')

df = df.merge(taxiZone,on='zone', how='inner')
df

diwaliDict = {}
for index, row in df.iterrows():
    diwaliDict[row['LocationID']] = row.to_dict()
    
with open('diwaliDrop.json', 'w') as fp:
    json.dump(diwaliDict, fp)

# df.to_csv("diwaliDrop.csv", index=False)
# df.to_json('diwaliDrop.json', orient = 'records', index = 'false')

In [ ]:
pizza = datetime.date(2018, 10, 10)


pizza = allData[(allData.dropoff == pizza)]
pizza = pizza.rename(columns={'dropoff': 'date',
                             'dropoffLocID': 'LocationID'})
print(pizza.shape)
pizza.head()

In [ ]:
pizza = pizza.merge(taxiZone, how='inner', on='LocationID')
pizza.head()

In [ ]:
pizza['borough'].value_counts()

In [ ]:
pizza['zone'].value_counts()

In [ ]:
dfDiwali = diwaliDropoff['zone'].value_counts().reset_index()
dfDiwali.columns = ['zone', 'count']

dfPizza = pizza['zone'].value_counts().reset_index()
dfPizza.columns = ['zone', 'count']

In [ ]:
dfPizza[dfPizza.zone == 'SoHo']